In [1]:
%%time
import abem as ab
import numpy as np
frequency = 40.0 # frequency [Hz]
k = ab.frequency_to_wavenumber(frequency)

interior_points = np.array([[0.0000,  0.0000],
                           [0.0000,  0.5000],
                           [0.0000, -0.5000],
                           [0.5000,  0.0000]], dtype=np.float32)


# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#
solver = ab.InteriorHelmholtzSolverRAD(ab.sphere_rad())

boundary_condition = solver.dirichlet_boundary_condition()
boundary_condition.f[:] = np.sin(k * solver.centers[:,1])

boundary_incidence = ab.BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

interior_incident_phi = np.zeros(interior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("Test Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundary_condition = solver.neumann_boundary_condition()
centers = solver.geometry.centers()
normals = solver.geometry.normals()
for i in range(centers.shape[0]):
    z = centers[i, 1]
    n = -normals[i]
    boundary_condition.f[i] = k * np.cos(k*z) * n[1]
    
boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Dirichlet boundary condition, such that phi = sin(k/ sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundary_condition.alpha.fill(1.0)
boundary_condition.beta.fill(0.0)

zp = 0.25
for i in range(centers.shape[0]):
    r = centers[i, 0]
    z = centers[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (z - zp)**2)
    boundary_condition.f[i]   = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    boundary_incidence.phi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    n = -normals[i]
    drbdn = (r * n[0] + (z - zp) * n[1]) / rpq
    boundary_incidence.v[i] = drbdn * np.exp(1j * k * rpq) * (1j * k * rpq - 1.0) \
        / (4.0 * np.pi * rpq*rpq)
    
for i in range(interior_points.shape[0]):
    r = interior_points[i, 0]
    z = interior_points[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (zp - z)**2)
    interior_incident_phi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
       
boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)

Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 0.7306029426953007 (40.0 Hz)

index   Potential               Pressure                 Velocity                 Intensity

    1   6.6323e-01+0.0000e+00   0.0000e+00+2.0086e+02i   5.4837e-01-2.3986e-03i  -2.4088e-01
    2   6.4667e-01+0.0000e+00   0.0000e+00+1.9584e+02i   5.4439e-01-2.5957e-03i  -2.5417e-01
    3   6.1291e-01+0.0000e+00   0.0000e+00+1.8562e+02i   5.2847e-01-2.3708e-03i  -2.2003e-01
    4   5.6148e-01+0.0000e+00   0.0000e+00+1.7004e+02i   5.0055e-01-1.9691e-03i  -1.6742e-01
    5   4.9228e-01+0.0000e+00   0.0000e+00+1.4909e+02i   4.5461e-01-1.5855e-03i  -1.1819e-01
    6   4.0551e-01+0.0000e+00   0.0000e+00+1.2281e+02i   3.8680e-01-1.1615e-03i  -7.1322e-02
    7   3.0276e-01+0.0000e+00   0.0000e+00+9.1690e+01i   2.9807e-01-6.8713e-04i  -3.1501e-02
    8   1.8738e-01+0.0000e+00   0.0000e+00+5.6748e+01i   1.8775e-01-4.4915e-04i  -1.2744e-02
    9   6.3520e-02+0.0